**Alur Kerja:**
1.  **Setup & Pemuatan Data:** Impor *library* dan muat data yang sudah diproses.
2.  **Definisi Fitur (Anti-Leakage):** Mendefinisikan *input* (X) dan *target* (y) dengan benar, menghindari *data leakage*.
3.  **Pembuatan Pipeline:** Membuat *pipeline* Scikit-learn untuk *scaling* dan *modeling*.
4.  **Pelatihan & Logging (MLflow):** Melatih model dan mencatat (log) metrik serta model ke MLflow.
5.  **Evaluasi:** Menganalisis `classification_report` dan `confusion_matrix`.

In [ ]:
%pip install mlflow xgboost scikit-learn==1.6.1

import mlflow
import mlflow.xgboost
import sklearn
import xgboost as xgb

print(f"Versi MLflow: {mlflow.__version__}")
print(f"Versi Scikit-learn: {sklearn.__version__}")
print(f"Versi XGBoost: {xgb.__version__}")

Versi MLflow: 3.5.1
Versi Scikit-learn: 1.6.1
Versi XGBoost: 3.1.1


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, recall_score, accuracy_score, precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import mlflow
import mlflow.xgboost
import joblib
import os
import warnings
from datetime import datetime

# Mengatur opsi tampilan dan peringatan
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

print(f"Semua library berhasil di-import. Waktu: {datetime.now()}")
print(f"Versi MLflow: {mlflow.__version__}")
print(f"Versi Scikit-learn: {sklearn.__version__}")
print(f"Versi XGBoost: {xgb.__version__}")

Semua library berhasil di-import. Waktu: 2025-10-30 10:07:23.419420
Versi MLflow: 3.5.1
Versi Scikit-learn: 1.6.1
Versi XGBoost: 3.1.1


## 1. Pemuatan & Inspeksi Data

Memuat dataset `processed_dataset.csv` dan melakukan verifikasi awal. Kita perlu memastikan:
1.  Data ter-load dengan benar.
2.  Tidak ada *missing values*.
3.  Tipe data (dtypes) sudah sesuai.

In [ ]:
file_path = 'processed_dataset.csv'

try:
    df = pd.read_csv(file_path)
    print(f"Dataset '{file_path}' berhasil dimuat.")
    print("--------------------------------------------------")

    # Menampilkan informasi ringkas (dtypes, non-null counts)
    print("Tampilan df.info():")
    df.info()

    print("\n--------------------------------------------------")
    # Menampilkan 5 baris pertama
    print("Tampilan df.head():")
    print(df.head())

except FileNotFoundError:
    print(f"ERROR: File '{file_path}' tidak ditemukan.")
    print("Pastikan file tersebut berada di folder yang sama dengan notebook Anda.")
except Exception as e:
    print(f"ERROR: Terjadi kesalahan saat memuat data: {e}")

✅ Dataset 'processed_dataset.csv' berhasil dimuat.
--------------------------------------------------
Tampilan df.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   UDI                               10000 non-null  int64  
 1   Product ID                        10000 non-null  object 
 2   Type                              10000 non-null  object 
 3   Air temperature [K]               10000 non-null  float64
 4   Process temperature [K]           10000 non-null  float64
 5   Rotational speed [rpm]            10000 non-null  float64
 6   Torque [Nm]                       10000 non-null  float64
 7   Tool wear [min]                   10000 non-null  int64  
 8   Target                            10000 non-null  int64  
 9   Type_Encoded                      10000 non-null  int64  
 10  Failure_H

## 2. Definisi Fitur & Target (Anti-Leakage)

Ini adalah langkah paling penting untuk memperbaiki kualitas model.

1.  **Target (y):** Kolom `Target` (0 atau 1).
2.  **Fitur (X):** Kolom-kolom yang berisi data operasional/sensor.
3.  **Anti-Leakage:** Kita **membuang** semua kolom `Failure_...` dari X, karena itu adalah *data leakage* (contekan). Model harus memprediksi kegagalan *sebelum* kita tahu *tipe* kegagalannya.

Kita juga akan melakukan *split* data (80% training, 20% testing).

In [ ]:
# --- 1. Definisi Fitur dan Target ---

# Ini tuh data leakage. GABOLEH ada di fitur X.
leaky_columns = [col for col in df.columns if 'Failure_' in col]

# Ini adalah kolom identitas atau kolom yang sudah di-enkoding/di-bin
# (Type sudah ada Type_Encoded, RPM dan Tool Wear lebih baik numerik aslinya)
redundant_columns = ['UDI', 'Product ID', 'Type', 'Tool_Wear_Bin', 'RPM_Bin']

# Target kita
TARGET_COLUMN = 'Target'

# --- 2. Membuat Daftar Fitur Final ---

# Fitur (X) adalah semua kolom DIKURANGI target, leakage, dan redundant
feature_columns = [
    col for col in df.columns
    if col not in [TARGET_COLUMN] + leaky_columns + redundant_columns
]

# kita bersihin nama kolom agar lebih mudah (menghilangkan spasi, [], dll.)
# Contoh: 'Air temperature [K]' -> 'Air_temperature_K'
original_columns = feature_columns + [TARGET_COLUMN]
new_column_names = {}

for col in original_columns:
    new_name = str(col).strip().replace(' ', '_').replace('[', '').replace(']', '').replace('(', '').replace(')', '').replace('/', '_')
    new_column_names[col] = new_name

# Terapkan ke DataFrame
df_model = df[original_columns].rename(columns=new_column_names)

# Update nama fitur dan target
feature_columns = [new_column_names[col] for col in feature_columns]
TARGET_COLUMN = new_column_names[TARGET_COLUMN]


print("--- Definisi Fitur dan Target ---")
print(f"Target (y): {TARGET_COLUMN}")
print(f"Fitur (X) yang akan digunakan ({len(feature_columns)}):")
print(feature_columns)

# --- 3. Split Data ---
X = df_model[feature_columns]
y = df_model[TARGET_COLUMN]

# Split data: 80% train, 20% test
# stratify=y penting untuk data imbalanced, agar proporsi 0 dan 1 sama di train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("\n--- Hasil Split Data ---")
print(f"Ukuran X_train: {X_train.shape}")
print(f"Ukuran X_test:  {X_test.shape}")

print("\nDistribusi 'Target' di y_train (untuk cek stratify):")
print(y_train.value_counts(normalize=True))
print("\nDistribusi 'Target' di y_test (untuk cek stratify):")
print(y_test.value_counts(normalize=True))

--- Definisi Fitur dan Target ---
✅ Target (y): Target
✅ Fitur (X) yang akan digunakan (6):
['Air_temperature_K', 'Process_temperature_K', 'Rotational_speed_rpm', 'Torque_Nm', 'Tool_wear_min', 'Type_Encoded']

--- Hasil Split Data ---
Ukuran X_train: (8000, 6)
Ukuran X_test:  (2000, 6)

Distribusi 'Target' di y_train (untuk cek stratify):
Target
0    0.966125
1    0.033875
Name: proportion, dtype: float64

Distribusi 'Target' di y_test (untuk cek stratify):
Target
0    0.966
1    0.034
Name: proportion, dtype: float64


## 3. Pembuatan Pipeline & Training (MLflow)

Kita akan membuat `Pipeline` Sklearn pake 2 langkah:
1.  `StandardScaler`: Melakukan *scaling* pada 6 fitur kita.
2.  `XGBClassifier`: Model *machine learning* kita.

Kita akan melatih *pipeline* ini dan mencatat (log) **keseluruhan pipeline** ke MLflow.

In [ ]:
# --- 1. Setup MLflow Experiment ---
# Kita akan menggunakan folder 'mlruns' lokal (default)
experiment_name = "Predictive_Maintenance_v2"
mlflow.set_experiment(experiment_name)

# Dapatkan ID eksperimen
try:
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
except AttributeError:
    # Jika eksperimen baru saja dibuat, 'None' mungkin dikembalikan
    experiment_id = mlflow.create_experiment(experiment_name)

print(f"--- Setup MLflow ---")
print(f"Experiment Name: {experiment_name}")
print(f"Experiment ID:   {experiment_id}")
print(f"Tracking URI:    'mlruns' (lokal)")

# --- 2. Buat Pipeline ---
# Kita gunakan parameter dasar untuk XGBoost terlebih dahulu
# 'scale_pos_weight' PENTING untuk data imbalanced
# (Jumlah 'No Failure' / Jumlah 'Failure' -> 96.6 / 3.4 ≈ 28)
# Mari kita hitung secara dinamis
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Scale Pos Weight (untuk Imbalance): {scale_pos_weight:.2f}")

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        random_state=42,
        scale_pos_weight=scale_pos_weight  # Memberi 'bobot' lebih pada kelas 'Failure'
    ))
])

# --- 3. Latih dan Log dengan MLflow ---
run_name = f"XGB_Pipeline_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

try:
    with mlflow.start_run(run_name=run_name, experiment_id=experiment_id) as run:
        print(f"\n--- Memulai MLflow Run ---")
        print(f"Run Name: {run_name}")
        print(f"Run ID:   {run.info.run_id}")

        # Latih pipeline
        print("... Melatih model (Pipeline)...")
        pipeline.fit(X_train, y_train)
        print("✅ Model (Pipeline) berhasil dilatih.")

        # Log parameter
        mlflow.log_param("model_type", "XGBClassifier")
        mlflow.log_param("scaler", "StandardScaler")
        mlflow.log_param("scale_pos_weight", scale_pos_weight)
        mlflow.log_param("features", feature_columns)

        # Log model (pipeline)
        # Ini adalah bagian terpentING untuk tim Back End
        print("... Menyimpan pipeline ke MLflow...")
        mlflow.sklearn.log_model(
            sk_model=pipeline,
            artifact_path="model_pipeline",  # Ini nama folder di dalam artifact
            input_example=X_train.head()     # Ini membantu MLflow menentukan schema
        )

        print("✅ Pipeline (Scaler + Model) berhasil di-log ke MLflow.")
        print(f"✅ Selesai! Buka UI MLflow (ketik 'mlflow ui' di terminal) untuk melihat run ini.")

except Exception as e:
    print(f"❌ ERROR saat training atau logging ke MLflow: {e}")

2025/10/30 10:07:23 INFO mlflow.tracking.fluent: Experiment with name 'Predictive_Maintenance_v2' does not exist. Creating a new experiment.


--- Setup MLflow ---
Experiment Name: Predictive_Maintenance_v2
Experiment ID:   549740798228718228
Tracking URI:    'mlruns' (lokal)
Scale Pos Weight (untuk Imbalance): 28.52

--- Memulai MLflow Run ---
Run Name: XGB_Pipeline_20251030_100723
Run ID:   3d0036ef207643699d8837a4db8e9e68
... Melatih model (Pipeline)...


2025/10/30 10:07:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Model (Pipeline) berhasil dilatih.
... Menyimpan pipeline ke MLflow...
✅ Pipeline (Scaler + Model) berhasil di-log ke MLflow.
✅ Selesai! Buka UI MLflow (ketik 'mlflow ui' di terminal) untuk melihat run ini.


## 4. Evaluasi Model & Logging Metrik

Kita sekarang akan menggunakan `pipeline` yang sudah dilatih untuk membuat prediksi pada `X_test`.

Fokus utama kita adalah **Recall** pada `Target = 1`. Kita ingin tahu: dari semua kegagalan yang *sebenarnya* terjadi, berapa banyak yang berhasil dideteksi oleh model?

Kita juga akan mencatat (log) metrik ini ke MLflow *run* kita sebelumnya untuk kelengkapan.

In [ ]:
# --- 1. Dapatkan Run ID dari output Langkah 4 ---
# Ganti ini dengan Run ID masing-masing! 
RUN_ID_ANDA = "3d0036ef207643699d8837a4db8e9e68"  # <-- GANTI INI DENGAN RUN ID masing2

# --- 2. Buat Prediksi ---
print("--- Membuat Prediksi ---")
try:
    # 'pipeline' adalah variabel yang sudah kita latih di Langkah 4
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1] # Probabilitas kegagalan
    print("✅ Prediksi pada X_test berhasil.")
except Exception as e:
    print(f"❌ Gagal membuat prediksi: {e}")

# --- 3. Hitung Metrik ---
accuracy = accuracy_score(y_test, y_pred)
# Kita fokus pada kelas '1' (Failure)
recall = recall_score(y_test, y_pred, pos_label=1)
precision = precision_score(y_test, y_pred, pos_label=1)

print("\n--- Metrik Evaluasi (Jujur) ---")
print(f"Akurasi:   {accuracy:.4f}")
print(f"Recall (1):  {recall:.4f}  <-- FOKUS UTAMA KITA")
print(f"Precision (1): {precision:.4f}")

# --- 4. Tampilkan Laporan Rinci ---
print("\n--- Laporan Klasifikasi ---")
# 0 = No Failure, 1 = Failure
print(classification_report(y_test, y_pred, target_names=['0 (No Failure)', '1 (Failure)']))

print("\n--- Confusion Matrix ---")
print("               Prediksi (0)   Prediksi (1)")
cm = confusion_matrix(y_test, y_pred)
print(f"Aktual (0)    {cm[0][0]:<12} {cm[0][1]:<12}")
print(f"Aktual (1)    {cm[1][0]:<12} {cm[1][1]:<12}  <-- (True Positive)")


# --- 5. Log Metrik ini ke MLflow Run yang tadi ---
print("\n--- Logging Metrik ke MLflow ---")
try:
    with mlflow.start_run(run_id=RUN_ID_ANDA) as run: # Buka lagi run yang tadi
        print(f"✅ Membuka Run ID: {RUN_ID_ANDA}")

        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("recall_class_1", recall)
        mlflow.log_metric("precision_class_1", precision)

        # Simpan confusion matrix sebagai teks
        cm_text = f"               Prediksi (0)   Prediksi (1)\nAktual (0)    {cm[0][0]:<12} {cm[0][1]:<12}\nAktual (1)    {cm[1][0]:<12} {cm[1][1]:<12}"
        mlflow.log_text(cm_text, "confusion_matrix.txt")

        print("✅ Metrik evaluasi berhasil di-log ke MLflow.")

except Exception as e:
    print(f"❌ Gagal logging metrik ke MLflow: {e}")
    print("Pastikan RUN_ID_ANDA sudah benar.")

--- Membuat Prediksi ---
✅ Prediksi pada X_test berhasil.

--- Metrik Evaluasi (Jujur) ---
Akurasi:   0.9790
Recall (1):  0.7647  <-- FOKUS UTAMA KITA
Precision (1): 0.6667

--- Laporan Klasifikasi ---
                precision    recall  f1-score   support

0 (No Failure)       0.99      0.99      0.99      1932
   1 (Failure)       0.67      0.76      0.71        68

      accuracy                           0.98      2000
     macro avg       0.83      0.88      0.85      2000
  weighted avg       0.98      0.98      0.98      2000


--- Confusion Matrix ---
               Prediksi (0)   Prediksi (1)
Aktual (0)    1906         26          
Aktual (1)    16           52            <-- (True Positive)

--- Logging Metrik ke MLflow ---
✅ Membuka Run ID: 3d0036ef207643699d8837a4db8e9e68
✅ Metrik evaluasi berhasil di-log ke MLflow.


## 5. Simulasi Penggunaan oleh Back End (Deployment)

Ini adalah sel terpenting untuk tim Back End Anda.

Kita akan menunjukkan cara memuat **keseluruhan pipeline** (Scaler + Model) yang kita simpan di MLflow hanya dengan menggunakan `Run ID`-nya.

Model yang dimuat ini akan secara otomatis:
1.  Menerima data *raw* (input JSON).
2.  Menerapkan `StandardScaler` (yang sudah dilatih).
3.  Menjalankan `XGBClassifier` (yang sudah dilatih).
4.  Mengembalikan prediksi (0 atau 1).

In [ ]:
# --- 1. Tentukan Model URI ---
# Ini adalah 'alamat' dari pipeline model kita
# Ganti dengan Run ID jika berbeda!
run_id = "3d0036ef207643699d8837a4db8e9e68"
model_uri = f"runs:/{run_id}/model_pipeline"

print(f"--- Simulasi Back End ---")
print(f"Mencoba memuat model dari URI: {model_uri}\n")

try:
    # --- 2. Muat Model ---
    # MLflow akan memuat pipeline (scaler + model)
    # Kita menggunakan 'mlflow.pyfunc' yang merupakan wrapper standar
    loaded_pipeline = mlflow.pyfunc.load_model(model_uri)
    print("✅ Pipeline (Scaler + Model) berhasil dimuat.")

    # --- 3. Siapkan Data Input Baru (Contoh) ---
    # Ini mensimulasikan data JSON yang masuk ke API tim Back End
    # Strukturnya HARUS sama dengan X_train

    # [Air_temp, Process_temp, RPM, Torque, Tool_wear, Type_Encoded]
    sample_data = {
        'Air_temperature_K':       [298.1, 301.5, 302.0],
        'Process_temperature_K':   [308.6, 310.0, 311.5],
        'Rotational_speed_rpm':    [1551,  1800,  2000],
        'Torque_Nm':               [42.8,  55.0,  68.0],  # Torque tinggi
        'Tool_wear_min':           [0,     50,    220],   # Tool wear tinggi
        'Type_Encoded':            [2,     1,     0]      # Tipe M, L, H
    }

    # Konversi ke Pandas DataFrame (ini penting!)
    sample_df = pd.DataFrame(sample_data)
    print("\nContoh data input (raw) yang akan diprediksi:")
    print(sample_df)

    # --- 4. Buat Prediksi ---
    # 'loaded_pipeline' akan OTOMATIS menerapkan scaler
    # dan kemudian model XGBoost
    predictions = loaded_pipeline.predict(sample_df)

    print("\n--- ✅ HASIL PREDIKSI ---")
    print(predictions)
    print("---------------------------------")
    print("Analisis: Jika Anda melihat [0 1 1] atau [0 0 1], ini sukses!")
    print("(Baris ke-2 dan ke-3 memiliki nilai Torque & Tool Wear yang tinggi, meningkatkan kemungkinan 'Failure=1')")

except Exception as e:
    print(f"❌ ERROR: Gagal memuat atau memprediksi dengan model MLflow: {e}")

--- Simulasi Back End ---
Mencoba memuat model dari URI: runs:/3d0036ef207643699d8837a4db8e9e68/model_pipeline



✅ Pipeline (Scaler + Model) berhasil dimuat.

Contoh data input (raw) yang akan diprediksi:
   Air_temperature_K  Process_temperature_K  Rotational_speed_rpm  Torque_Nm  \
0              298.1                  308.6                  1551       42.8   
1              301.5                  310.0                  1800       55.0   
2              302.0                  311.5                  2000       68.0   

   Tool_wear_min  Type_Encoded  
0              0             2  
1             50             1  
2            220             0  
❌ ERROR: Gagal memuat atau memprediksi dengan model MLflow: Failed to enforce schema of data '   Air_temperature_K  Process_temperature_K  Rotational_speed_rpm  Torque_Nm  \
0              298.1                  308.6                  1551       42.8   
1              301.5                  310.0                  1800       55.0   
2              302.0                  311.5                  2000       68.0   

   Tool_wear_min  Type_Encoded  
0      

## 5. Simulasi Penggunaan oleh Back End (Perbaikan)

Kita mengalami *schema enforcement error*. Ini adalah fitur keamanan MLflow.

**Masalah:** Data `X_train` asli memiliki `Rotational_speed_rpm` sebagai `float` (desimal). Data sampel kita menyediakannya sebagai `int` (bulat).

**Perbaikan:** Kita akan secara eksplisit membuat data sampel sebagai `float` (misalnya `1551.0`) agar sesuai dengan skema model.

In [ ]:
# --- 1. Tentukan Model URI ---
# Ganti dengan Run ID jika berbeda!
run_id = "3d0036ef207643699d8837a4db8e9e68"
model_uri = f"runs:/{run_id}/model_pipeline"

print(f"--- Simulasi Back End (Perbaikan) ---")
print(f"Mencoba memuat model dari URI: {model_uri}\n")

try:
    # --- 2. Muat Model ---
    loaded_pipeline = mlflow.pyfunc.load_model(model_uri)
    print("✅ Pipeline (Scaler + Model) berhasil dimuat.")

    # --- 3. Siapkan Data Input Baru (FIXED) ---

    # [Air_temp, Process_temp, RPM, Torque, Tool_wear, Type_Encoded]
    sample_data = {
        'Air_temperature_K':       [298.1, 301.5, 302.0],
        'Process_temperature_K':   [308.6, 310.0, 311.5],
        # --- PERBAIKAN DI SINI: Tambahkan .0 ---
        'Rotational_speed_rpm':    [1551.0,  1800.0,  2000.0],
        'Torque_Nm':               [42.8,  55.0,  68.0],
        'Tool_wear_min':           [0,     50,    220],       # Ini int64, sudah benar
        'Type_Encoded':            [2,     1,     0]        # Ini int64, sudah benar
    }

    # Konversi ke Pandas DataFrame
    sample_df = pd.DataFrame(sample_data)
    print("\nContoh data input (raw) yang sudah diperbaiki:")
    print(sample_df)
    print("\nCek dtypes input:")
    print(sample_df.info()) # Kita cek dtypes kita

    # --- 4. Buat Prediksi ---
    predictions = loaded_pipeline.predict(sample_df)

    print("\n--- ✅ HASIL PREDIKSI ---")
    print(predictions)
    print("---------------------------------")

    # Analisis hasil yang diharapkan
    # [Data normal, Data risiko tinggi, Data risiko sangat tinggi]
    # Kita mengharapkan sesuatu seperti [0, 1, 1] atau [0, 0, 1]

    if list(predictions) == [0, 1, 1]:
        print("Analisis: Sukses! [0, 1, 1]. Model memprediksi 2 data berisiko tinggi sebagai 'Failure'.")
    elif list(predictions) == [0, 0, 1]:
         print("Analisis: Sukses! [0, 0, 1]. Model memprediksi 1 data berisiko tinggi sebagai 'Failure'.")
    elif list(predictions) == [0, 0, 0]:
         print("Analisis: [0, 0, 0]. Model memprediksi 'No Failure' untuk semua.")
    else:
        print(f"Analisis: Hasil prediksi {list(predictions)}")


except Exception as e:
    print(f"❌ ERROR: Gagal memuat atau memprediksi dengan model MLflow: {e}")

--- Simulasi Back End (Perbaikan) ---
Mencoba memuat model dari URI: runs:/3d0036ef207643699d8837a4db8e9e68/model_pipeline



✅ Pipeline (Scaler + Model) berhasil dimuat.

Contoh data input (raw) yang sudah diperbaiki:
   Air_temperature_K  Process_temperature_K  Rotational_speed_rpm  Torque_Nm  \
0              298.1                  308.6                1551.0       42.8   
1              301.5                  310.0                1800.0       55.0   
2              302.0                  311.5                2000.0       68.0   

   Tool_wear_min  Type_Encoded  
0              0             2  
1             50             1  
2            220             0  

Cek dtypes input:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Air_temperature_K      3 non-null      float64
 1   Process_temperature_K  3 non-null      float64
 2   Rotational_speed_rpm   3 non-null      float64
 3   Torque_Nm              3 non-null      float64
 4   Tool_wear_min     

# 6. Dokumentasi & Serah Terima Tim Back End

Proyek ini selesai. Model telah dilatih, dievaluasi (Recall 76.5%), dan dicatat (log) ke MLflow.

### Artefak yang Dihasilkan
1.  **Folder `mlruns`:** Berisi semua *data run*, metrik, dan *file* model. Ini adalah "database" model kita.
2.  **Run ID Model Terbaik:** `59fdd51a5fcf40ef867fa57ae75c9994`
3.  **Path Artefak:** `model_pipeline`

### Cara Menggunakan Model (Untuk Tim Back End)

Tim Back End dapat memuat dan menggunakan *pipeline* (Scaler + Model) ini hanya dengan 3 langkah di API (Flask/FastAPI) mereka:

**Prasyarat:**
Pastikan *environment* Python Anda memiliki:
```bash
pip install mlflow pandas "scikit-learn==1.6.1" "xgboost==3.0.1"

---
#### Kode API Contoh:
```python
import mlflow
import pandas as pd

# 1. TENTUKAN & MUAT MODEL (Lakukan sekali saat API start)
# Pastikan folder 'mlruns' berada di direktori yang sama
RUN_ID = "59fdd51a5fcf40ef867fa57ae75c9994"
MODEL_URI = f"runs:/{RUN_ID}/model_pipeline"

try:
    # Ini memuat Scaler + Model XGBoost
    loaded_model = mlflow.pyfunc.load_model(MODEL_URI)
    print("✅ Model Predictive Maintenance berhasil dimuat.")
except Exception as e:
    print(f"❌ Gagal memuat model: {e}")


# 2. BUAT FUNGSI PREDIKSI (Ini adalah endpoint API)
def predict_maintenance(json_input_data):
    """
    Menerima input JSON (bisa list of dict) dan mengembalikan prediksi.
    """
    # MLflow mengharapkan DataFrame
    data_df = pd.DataFrame(json_input_data)
    
    # MLflow/model akan otomatis menerapkan scaler dan memprediksi
    predictions = loaded_model.predict(data_df)
    
    # Konversi ke format JSON yang ramah
    return list(predictions)

# 3. CONTOH PENGGUNAAN
# Ini adalah data yang mungkin dikirim oleh Chatbot ke API
sample_data = [
    { # Data Normal
        'Air_temperature_K': 298.1,
        'Process_temperature_K': 308.6,
        'Rotational_speed_rpm': 1551.0, # HARUS float
        'Torque_Nm': 42.8,
        'Tool_wear_min': 0,
        'Type_Encoded': 2
    },
    { # Data Berisiko Tinggi
        'Air_temperature_K': 302.0,
        'Process_temperature_K': 311.5,
        'Rotational_speed_rpm': 2000.0, # HARUS float
        'Torque_Nm': 68.0,
        'Tool_wear_min': 220,
        'Type_Encoded': 0
    }
]

# Panggil fungsi prediksi
hasil = predict_maintenance(sample_data)
print(f"Hasil Prediksi: {hasil}") # Output yang diharapkan: [0, 1]
```